## Using BigQuery Embeddings in a RAG Architecture

### Overview

In this lab, you will load a dataset of text into BigQuery, and use BigQuery's ability to create text embeddings for each chunk of text.

You will then implement a Retrieval-Augmented Generation (RAG) architecture, which is used to generate an embedding of a user's query, find the most similar embeddings of data from our text dataset, and use those corresponding chunks of text to provide a language model relevant info to use when generating its response to the user's query.

#### What are embeddings?

Like all machine learning models, large language models must convert all inputs to numerical representations in order to work with them.

Part of the process of training a large language model includes guiding it to learn to represent **tokens** numerically. Tokens may be words like "glass" or pieces of words, like "un", "break" and "able" from the word "unbreakable". Language models learn to represent an individual token as a **vector embedding**, or string of numbers like [0.12232, 0.23442, 0.23242, ...] (using the embedding model `text-embedding-004` this would be a list 768 numbers long, so we'll stop there). We would refer to the length of this array as the number of dimensions of the embedding.

Multiple tokens can be combined into a single embedding, still with the same 768 dimensions. So that is why we can also generate embeddings for sentences, paragraphs, or even pages of text. Note that there is a limit, which in the case of `text-embedding-004` is a [token limit of 2,048 tokens before input text is truncated](https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-text-embeddings#get_text_embeddings_for_a_snippet_of_text).

These vector embeddings represent coordinates in an _embedding space_, with similar tokens appearing nearby one another. So just like in Algebra class, where you may have calculated the distance between two points of x and y coordinates like (0, 0) and (3, 3), we can calculate the distance between text embeddings. In this way, we can determine which content a language model considers to have similar meaning by the relatively smaller distances between the coordinates represented by their embeddings.

### Objective

Create text embeddings within BigQuery and use them in a RAG architecture to inform a language model's responses.

#### What you will learn

In this lab, you learn how to:

- Connect BigQuery to a Vertex AI embedding model.
- Use this model to create text embeddings and store them in a BigQuery table.
- Use this model to generate an embedding for a query and perform a similarity search on the embeddings using SQL in BigQuery.
- Code a Q&A system that retrieves relevant information to a query and uses Google Gemini to answer questions about it.


### Task 1. Create a Vertex AI connection to enable ML calls using BigQuery SQL

1. On the Google Cloud Console title bar, click the **Navigation Menu** (Navigation menu icon). Scroll to the Analytics section and click **BigQuery**.
2. In the Explorer pane of the BigQuery Console, click on the three dots next to your Project ID (`PROJECT_ID`) and then select **Create dataset**.
3. In the Create dataset dialog, set the Dataset ID to bbc_news and make sure the multi-region US location is selected and leave the other options at their default values.
4. Click **Create dataset**.
5. Next, you will create an external connection that allows you to use the models available at the Vertex AI API to generate embeddings. At the top of the Explorer pane (the pane containing your list of available BigQuery datasets) click the **+ Add** button. Select the **Connections to External data sources** button.

**Note**: If prompted, enable the **BigQuery API** and click on **Connections to External data sources**.

<div style="text-align: center;">
    <img width="80%" src="./images/a08_05-01.png">
</div>

6. In the **Connection type** dropdown, select the `Vertex AI remote models, remote functions and BigLake (Cloud Resource)`. Set the Connection ID to `Connection id`. Ensure the location is set to Multi-region US. Click the Create Connection.
7. In the **Explorer** pane, expand **External connections**, and select the `External connection id` connection you just created. In the **Connection info** pane, copy the `Service account id` property value to the clipboard. You will need to add permissions to this account to use Vertex AI and your BigQuery data.

<div style="text-align: center;">
    <img width="80%" src="./images/a08_05-02.png">
</div>

8. Click the **Navigation Menu** (Navigation menu icon) and go to **IAM & Admin** service. On the IAM page, click the **Grant Access** button. Paste the `service account id` into the `Principals` text box. Assign the **Vertex AI User** and **BigQuery Data Owner** roles to the account and click the Save button.

<div style="text-align: center;">
    <img width="80%" src="./images/a08_05-03.png">
</div>

9. Click the **Navigation Menu** (Navigation menu icon) and navigate to the **Vertex AI** service. Click the **Enable All Recommended APIs** button.
10. Navigate to **BigQuery** service. In the BigQuery code editor, run the following command to create a model in your bbc_news dataset that connects to a Vertex AI Text Embedding model.

```sql
CREATE OR REPLACE MODEL `bbc_news.Model Name`
  REMOTE WITH CONNECTION `us.Connection id`
  OPTIONS (ENDPOINT = 'text-embedding-004');
```

### Task 2. Create Text Embeddings using BigQuery 

In this task, you will create a table that contains articles from the Google BBC News public dataset as well as text embeddings that can be used for a vector similarity search.

1. In this task, you will create a table that contains articles from the Google BBC News public dataset as well as text embeddings that can be used for a vector similarity search.

```sql
SELECT * FROM `bigquery-public-data.bbc_news.fulltext` LIMIT 5;
```

2. Run the following SQL query. This creates a table named `Table Name` with the BBC News articles from the public dataset and text embeddings generated with the model named `Model Name` you created above. Notice, that the title and body fields are concatenated into a new field called content. The function ML.GENERATE_TEXT_EMBEDDING looks for a field named content and generates the embeddings from that field.

```sql
CREATE OR REPLACE TABLE
  `bbc_news.Table Name` AS (
  SELECT
    *
  FROM
    ML.GENERATE_TEXT_EMBEDDING( MODEL `bbc_news.Model Name`,
      (
      SELECT
        title,
        body,
        CONCAT(title, ": ", body) AS content
      FROM
        `bigquery-public-data.bbc_news.fulltext`
      WHERE
        LENGTH(body) > 0 AND LENGTH(title) > 0
     )
    ) WHERE ARRAY_LENGTH(text_embedding) > 0
    LIMIT 500
  );
```

**Note**: If the table had more than 5000 rows, you would use the following command to create a Vector Index. This won't work in this example because the table has about 500 rows. The code below is shown here as a resource for implementing this pattern on larger projects, but don't run it as part of this lab.

```sql
CREATE OR REPLACE VECTOR INDEX bbc_news_index
ON `bbc_news.Table Name`(text_embedding)
OPTIONS(index_type = 'IVF',
  distance_type = 'COSINE',
  ivf_options = '{"num_lists":500}')
```

3. Expand the bbc_news dataset and select the `Table Name` table.

4. Take a look at the schema. Note that the text_embedding field has a type of Float with a mode of Repeated. This is how text embeddings should be represented in BigQuery, whether you generate them in BigQuery as we have done or if you generate them externally and upload them to a BigQuery table. Click on the **Preview** button to see a few rows of the data.


### Task 3. Run a Vector Search using BigQuery SQL

Now you will query the table for relevant news articles. Run the below query to return articles about The US Economy.

**Note**: An embedding is created for the string "The US Economy". That embedding is used in a similarity search and the rows for the top 5 articles with embeddings closest to the query's embedding are returned. Examine the results and verify the returned articles are related to the query.

```sql
SELECT query.query, base.title, base.body
FROM VECTOR_SEARCH(
  TABLE `bbc_news.Table Name`, 'text_embedding',
  (
  SELECT text_embedding, content AS query
  FROM ML.GENERATE_TEXT_EMBEDDING(
    MODEL `bbc_news.Model Name`,
    (SELECT 'The US Economy' AS content))
  ),
  top_k => 5, options => '{"fraction_lists_to_search": 0.01}')
```

2. Change the search string and try a couple more queries. Search for things like: "Political unrest in Africa", "developments in the technology sector", and "uplifting human success stories".

### Task 4. Program a Q&A System using BigQuery and Gemini

1. Navigate to Vertex AI service and select Colab Enterprise from the Notebooks section of the navigation pane.
2. Select the region as `Lab Region` from dropdown and then click on **+ Create a New Notebook** button.
    - Enter the following code in the first cell and run it.
    - Once cell execution done, click on RESTART SESSION.

```python
!pip install --upgrade --user google-cloud-aiplatform google-cloud-bigquery
```

In [ ]:
# 3. Add another code cell and paste the following code. It initializes Vertex AI.
# Get project ID
PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "Lab Region" # @param {type:"string"}
print(PROJECT_ID)

from google.cloud import aiplatform
aiplatform.init(project=PROJECT_ID, location=LOCATION)

print("Initialized")

In [ ]:
# 4. Add the following function in another code cell. This function allows you to pass a prompt to the Gemini LLM and get an answer.

import vertexai
from vertexai.generative_models import GenerativeModel, Part

def answer_question_gemini(prompt):
  model = GenerativeModel("gemini-pro")
  response = model.generate_content(
    prompt,
    generation_config={
        "max_output_tokens": 8192,
        "temperature": 0.5,
        "top_p": 0.5,
        "top_k": 10,
    },
  stream=False,
  )
  try:
    return response.text
  except:
    print("An Error Ocuured Cleaning the Data")
    return "An Error Ocuured Cleaning the Data"

In [ ]:
# 5. Add the following function to another new code cell. This function takes as an argument the question the user asks, 
# uses it to create an embedding, and uses a similarity search to find the top 5 articles most relevant to the question. 
# After it runs the query, it concatenates the 5 five articles together into a single string and returns that string.

def run_search(question):
  from google.cloud import bigquery

  client = bigquery.Client()

  sql = """
      SELECT query.query, base.title, base.body
      FROM VECTOR_SEARCH(
      TABLE `bbc_news.Table Name`, 'text_embedding',
      (
      SELECT text_embedding, content AS query
      FROM ML.GENERATE_TEXT_EMBEDDING(MODEL `bbc_news.Model Name`,
          (SELECT @question AS content))),
      top_k => 5)
      """

  job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("question", "STRING", question),
    ]
  )

  query_job = client.query(sql, job_config=job_config)

  data = ""
  for row in query_job:
    data += row.body + "\n"

  return data

In [ ]:
# 6. Again, add another code cell and paste the following function. 
# This function uses the question and the data retrieved from BigQuery to build a prompt that will be passed to the mode.

def build_prompt(data, question):
  prompt = """
    Instructions: Answer the question using the following Context.

    Context: {0}

    Question: {1}
  """.format(data, question)
  return prompt

In [ ]:
# 7. Add another code cell and paste the following function. This is helper function.

from IPython.core.display import display, HTML

def answer_question(question):

  data = run_search(question)
  display("Retrieved Data:")
  display(data)
  display(" . . . ")
  prompt = build_prompt(data, question)
  answer_gemini = answer_question_gemini(prompt)

  return answer_gemini

### Task 5. Test the Q&A System

In [ ]:
# 1. Add one more code cell to your notebook and enter the following code that asks a question and returns the answer.

QUESTION = "Tell me about the US Economy"

answer_gemini = answer_question(QUESTION)
display("User Question:")
display(QUESTION)
display("--------------------------------")
display("Gemini Answer:")
display(answer_gemini)

2. Try some other questions like, "What's happening in sports?".
3. See what happens if you ask a question that has no relevant news articles. For example, "What it a good recipe for bouillabaisse?".